In [68]:
import pandas as pd
import numpy as np
import scipy
import array as arr
from scipy.stats import mannwhitneyu

## Mann–Whitney U test

El test **Mann-Whitney U** se usa a menudo para probar si hay una diferencia entre dos categorías en una variable ordinal (Las variables ordinales, a diferencia de las nominales, sí pueden ser ordenadas de forma jerárquica).

Aunque a menudo se piensa que se compara la mediana de cada categoría, en realidad se comparan los rangos medios de cada grupo.

Los rangos se determinan simplemente ordenando primero todos los puntajes en la variable ordinal, luego el puntaje más bajo obtiene el rango 1, el siguiente rango 2, etc. El rango más alto posible es, por lo tanto, el número total de casos. Si dos o más puntajes son iguales, se usa el promedio de los rangos que hubieran obtenido. Entonces, si, por ejemplo, el cuarto puntaje es un 9, el quinto es un 9 y el sexto es un 9, entonces el rango para el puntaje cuatro, cinco y seis será cada uno (4 + 5 + 6) / 3 = 5.

In [69]:
def is_unique(s):
    a = s.to_numpy() # s.values (pandas<0.24)
    return (a[0] == a).all()

In [70]:
Laptop = pd.read_csv('./Data/Laptop/tournament-20210524-195728-Acquisition-A-domain.log.csv', sep=';')
Grocery = pd.read_csv('./Data/Grocery/tournament-20210524-220009-Grocery-A-domain.log.csv', sep=';')
ItexCypress = pd.read_csv('./Data/Itex-Cypress/tournament-20210525-093515-ItexvsCypress-A-domain.log.csv', sep=';')
Travel = pd.read_csv('./Data/Travel/tournament-20210525-122752-Travel-A-domain.log.csv', sep=';')
Evaluation = pd.read_csv('./Data/Evaluation/tournament-20210525-184830-Laptop-A-domain.log.csv', sep=';')

### Limpieza de columnas

In [71]:
contents = ['boa-negotiator.boaframework.offeringstrategy.', 'negotiator.boaframework.acceptanceconditions.', 'negotiator.boaframework.opponentmodel.', 'negotiator.boaframework.omstrategy.']
for content in contents:
    Laptop['Agent 1'] = Laptop['Agent 1'].str.replace(content,'')
    Grocery['Agent 1'] = Grocery['Agent 1'].str.replace(content,'')
    ItexCypress['Agent 1'] = ItexCypress['Agent 1'].str.replace(content,'')
    Travel['Agent 1'] = Travel['Agent 1'].str.replace(content,'')
    
    Laptop['Agent 2'] = Laptop['Agent 1'].str.replace(content,'')
    Grocery['Agent 2'] = Grocery['Agent 1'].str.replace(content,'')
    ItexCypress['Agent 2'] = ItexCypress['Agent 1'].str.replace(content,'')
    Travel['Agent 2'] = Travel['Agent 1'].str.replace(content,'')
#Columna: Agent 1
Laptop['Agent 1'] = Laptop['Agent 1'].str.replace('-','|').str.split('@').str[0]
Grocery['Agent 1'] = Grocery['Agent 1'].str.replace('-','|').str.split('@').str[0]
ItexCypress['Agent 1'] = ItexCypress['Agent 1'].str.replace('-','|').str.split('@').str[0]
Travel['Agent 1'] = Travel['Agent 1'].str.replace('-','|').str.split('@').str[0]
#Columna: Agent 2
Laptop['Agent 2'] = Laptop['Agent 1'].str.replace('-','|').str.split('@').str[0]
Grocery['Agent 2'] = Grocery['Agent 1'].str.replace('-','|').str.split('@').str[0]
ItexCypress['Agent 2'] = ItexCypress['Agent 1'].str.replace('-','|').str.split('@').str[0]
Travel['Agent 2'] = Travel['Agent 1'].str.replace('-','|').str.split('@').str[0]

### Agentes

In [72]:
agents = Laptop['Agent 1'].drop_duplicates().tolist()
agents

['anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t',
 'anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t',
 'anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t',
 'anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t',
 'anac2011.NiceTitForTat_Offering|anac2012.AC_CUHKAgent|PerfectModel|BestBid_t',
 'anac2011.NiceTitForTa

### Laptop

In [73]:
dictionary1 = {}
mediana1 = {}
mejor1 = {}
for agent in agents:
    a = Laptop[Laptop['Agent 1'] == agent]['Utility 1']
    b = Laptop[Laptop['Agent 2'] == agent]['Utility 2']
    frames = [a, b]
    dictionary1[agent] = pd.concat(frames)
    mediana1[agent] = pd.concat(frames).median()
    mejor1[agent] = 0

In [74]:
compare = []
for key1 in dictionary1:
    noiguales = 0
    iguales = 0
    for key2 in dictionary1:
        if key1 != key2:
            if is_unique(dictionary1[key1]) == False and is_unique(dictionary1[key2]) == False:
                S, p = mannwhitneyu(dictionary1[key1], dictionary1[key2], use_continuity=False, alternative = 'two-sided')
                if p < 0.05:
                    noiguales += 1
                    if key1 not in compare:
                        if mediana1[key1] != mediana1[key2]:
                            if mediana1[key1] > mediana1[key2]:
                                 mejor1[key1] += 1
                else:
                    iguales += 1
    compare.append(key1)
    print(key1, "Igual|No igual =>", str(iguales) + "|" + str(noiguales))

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t Igual|No igual => 12|18
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 12|18
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 12|18
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t Igual|No igual => 6|24
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t Igual|No igual => 7|23
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t Igual|No igual => 7|23
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t Igual|No igual => 7|23
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 12|18
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 13|17
anac2013.InoxAgent_O

In [75]:
for key in mejor1:
    print(key, '=>', mejor1[key])

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 18
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 18
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t => 7
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t => 7
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t => 7
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 15
anac2011.NiceTitForTat_Offering|anac2012.AC_CUHKAgent|PerfectModel|BestBid_t => 0
anac20

### Grocery

In [76]:
dictionary2 = {}
mediana2 = {}
mejor2 = {}
for agent in agents:
    a = Grocery[Grocery['Agent 1'] == agent]['Utility 1']
    b = Grocery[Grocery['Agent 2'] == agent]['Utility 2']
    frames = [a, b]
    dictionary2[agent] = pd.concat(frames)
    mediana2[agent] = pd.concat(frames).median()
    mejor2[agent] = 0

In [77]:
compare = []
for key1 in dictionary2:
    noiguales = 0
    iguales = 0
    for key2 in dictionary2:
        if key1 != key2:
            if is_unique(dictionary2[key1]) == False and is_unique(dictionary2[key2]) == False:
                S, p = mannwhitneyu(dictionary2[key1], dictionary2[key2], use_continuity=False, alternative = 'two-sided')
                if p < 0.05:
                    noiguales += 1
                    if key1 not in compare:
                        if mediana2[key1] != mediana2[key2]:
                            if mediana2[key1] > mediana2[key2]:
                                 mejor2[key1] += 1
                else:
                    iguales += 1
    compare.append(key1)
    print(key1, "Igual|No igual =>", str(iguales) + "|" + str(noiguales))

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t Igual|No igual => 9|21
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 9|21
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 10|20
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t Igual|No igual => 3|27
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t Igual|No igual => 6|24
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t Igual|No igual => 6|24
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t Igual|No igual => 7|23
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 9|21
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 9|21
anac2013.InoxAgent_Offer

In [78]:
for key in mejor2:
    print(key, '=>', mejor2[key])

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 17
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 15
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t => 8
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t => 9
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t => 9
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 15
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 15
anac2011.NiceTitForTat_Offering|anac2012.AC_CUHKAgent|PerfectModel|BestBid_t => 0
anac20

### Itex-Cypress

In [79]:
dictionary3 = {}
mediana3 = {}
mejor3 = {}
for agent in agents:
    a = ItexCypress[ItexCypress['Agent 1'] == agent]['Utility 1']
    b = ItexCypress[ItexCypress['Agent 2'] == agent]['Utility 2']
    frames = [a, b]
    dictionary3[agent] = pd.concat(frames)
    mediana3[agent] = pd.concat(frames).median()
    mejor3[agent] = 0

In [80]:
compare = []
for key1 in dictionary3:
    noiguales = 0
    iguales = 0
    for key2 in dictionary3:
        if key1 != key2:
            if is_unique(dictionary3[key1]) == False and is_unique(dictionary3[key2]) == False:
                S, p = mannwhitneyu(dictionary3[key1], dictionary3[key2], use_continuity=False, alternative = 'two-sided')
                if p < 0.05:
                    noiguales += 1
                    if key1 not in compare:
                        if mediana3[key1] != mediana3[key2]:
                            if mediana3[key1] > mediana3[key2]:
                                 mejor3[key1] += 1
                else:
                    iguales += 1
    compare.append(key1)
    print(key1, "Igual|No igual =>", str(iguales) + "|" + str(noiguales))

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t Igual|No igual => 6|16
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 4|18
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 1|21
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t Igual|No igual => 0|0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t Igual|No igual => 0|0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t Igual|No igual => 0|0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t Igual|No igual => 0|0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 6|16
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 6|16
anac2013.InoxAgent_Offering_O

In [81]:
for key in mejor3:
    print(key, '=>', mejor3[key])

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 0
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 0
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t => 19
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 0
anac2011.NiceTitForTat_Offering|anac2012.AC_CUHKAgent|PerfectModel|BestBid_t => 0
anac2011.Ni

### Travel

In [82]:
dictionary4 = {}
mediana4 = {}
mejor4 = {}
for agent in agents:
    a = Travel[Travel['Agent 1'] == agent]['Utility 1']
    b = Travel[Travel['Agent 2'] == agent]['Utility 2']
    frames = [a, b]
    dictionary4[agent] = pd.concat(frames)
    mediana4[agent] = pd.concat(frames).median()
    mejor4[agent] = 0

In [83]:
compare = []
for key1 in dictionary4:
    noiguales = 0
    iguales = 0
    for key2 in dictionary4:
        if key1 != key2:
            if is_unique(dictionary4[key1]) == False and is_unique(dictionary4[key2]) == False:
                S, p = mannwhitneyu(dictionary4[key1], dictionary4[key2], use_continuity=False, alternative = 'two-sided')
                if p < 0.05:
                    noiguales += 1
                    if key1 not in compare:
                        if mediana4[key1] != mediana4[key2]:
                            if mediana4[key1] > mediana4[key2]:
                                 mejor4[key1] += 1
                else:
                    iguales += 1
    compare.append(key1)
    print(key1, "Igual|No igual =>", str(iguales) + "|" + str(noiguales))

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t Igual|No igual => 26|4
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 7|23
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 24|6
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t Igual|No igual => 18|12
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t Igual|No igual => 24|6
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t Igual|No igual => 18|12
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t Igual|No igual => 24|6
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t Igual|No igual => 24|6
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t Igual|No igual => 16|14
anac2013.InoxAgent_Off

In [84]:
for key in mejor4:
    print(key, '=>', mejor4[key])

anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 0
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 0
anac2012.TheNegotiatorReloaded_Offering|anac2010.AC_AgentK|PerfectModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2012.AC_CUHKAgent|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|PerfectModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|NashFrequencyModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|other.AC_Uncertain|ScalableBayesianModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|PerfectModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|NashFrequencyModel|BestBid_t => 0
anac2013.InoxAgent_Offering_OneIssue|anac2010.AC_AgentK|ScalableBayesianModel|BestBid_t => 0
anac2011.NiceTitForTat_Offering|anac2012.AC_CUHKAgent|PerfectModel|BestBid_t => 0
anac2011.Nic

In [103]:
Travel['Run time (s)'] = Travel['Run time (s)'].str.replace(',','.').astype(float)
verify = Travel.sort_values(by='Run time (s)', ascending=False)
verify.head()

,Run time (s),Round,Exception,deadline,Agreement,Discounted,#agreeing,min.util.,max.util.,Dist. to Pareto,...,Disc. Util. 1,Disc. Util. 2,Perceived. Util. 1,Perceived. Util. 2,User Bother 1,User Bother 2,User Util. 1,User Util. 2,Profile 1,Profile 2
674,1193.388,46,NaN,60rounds,Yes,Yes,2,"0,51850","0,68247","0,04417",...,0.518501,0.682474,0.518501,0.682474,0.0,0.0,0.518501,0.682474,Travel-A-prof1.xml,Travel-A-prof2.xml
640,142.634,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
637,140.996,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
646,140.761,61,NaN,60rounds,Yes,Yes,2,"0,50403","0,72164","0,02261",...,0.721643,0.504034,0.721643,0.504034,0.0,0.0,0.721643,0.504034,Travel-A-prof1.xml,Travel-A-prof2.xml
631,135.268,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml


In [104]:
pd.set_option('display.max_rows', verify.shape[0]+1)
verify

,Run time (s),Round,Exception,deadline,Agreement,Discounted,#agreeing,min.util.,max.util.,Dist. to Pareto,...,Disc. Util. 1,Disc. Util. 2,Perceived. Util. 1,Perceived. Util. 2,User Bother 1,User Bother 2,User Util. 1,User Util. 2,Profile 1,Profile 2
674,1193.388,46,NaN,60rounds,Yes,Yes,2,"0,51850","0,68247","0,04417",...,0.518501,0.682474,0.518501,0.682474,0.0,0.0,0.518501,0.682474,Travel-A-prof1.xml,Travel-A-prof2.xml
640,142.634,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
637,140.996,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
646,140.761,61,NaN,60rounds,Yes,Yes,2,"0,50403","0,72164","0,02261",...,0.721643,0.504034,0.721643,0.504034,0.0,0.0,0.721643,0.504034,Travel-A-prof1.xml,Travel-A-prof2.xml
631,135.268,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
634,133.212,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
460,121.851,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
454,120.908,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
457,120.644,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml
547,115.612,61,NaN,60rounds,No,Yes,1,"0,37677","0,37677","0,75194",...,0.376773,0.376773,0.000000,0.000000,0.0,0.0,0.376773,0.376773,Travel-A-prof1.xml,Travel-A-prof2.xml


### Modificación modelo

In [85]:
contents = ['boa-negotiator.boaframework.offeringstrategy.', 'negotiator.boaframework.acceptanceconditions.', 'C..Desarrollo.Master.OpponentModel.out.production.OpponentModel.es.upv.opponentmodel.', 'negotiator.boaframework.omstrategy.']
for content in contents:
    Evaluation['Agent 1'] = Evaluation['Agent 1'].str.replace(content,'')    
    Evaluation['Agent 2'] = Evaluation['Agent 1'].str.replace(content,'')
#Columna: Agent 1
Evaluation['Agent 1'] = Evaluation['Agent 1'].str.replace('-','|').str.split('@').str[0]
#Columna: Agent 2
Evaluation['Agent 2'] = Evaluation['Agent 1'].str.replace('-','|').str.split('@').str[0]

In [86]:
agents1 = Evaluation['Agent 1'].drop_duplicates().tolist()
agents1

['anac2012.CUHKAgent_Offering|other.AC_Uncertain|PracticeSimpleFrequencyModel_delta|BestBid_t',
 'anac2012.CUHKAgent_Offering|other.AC_Uncertain|JaidenMeidenModel_delta_multa|BestBid_t',
 'TheFawkes',
 'KLH',
 'Atlas3',
 'PonPokoAgent']

In [87]:
dictionary5 = {}
mediana5 = {}
mejor5 = {}
for agent in agents1:
    a = Evaluation[Evaluation['Agent 1'] == agent]['Utility 1']
    b = Evaluation[Evaluation['Agent 2'] == agent]['Utility 2']
    frames = [a, b]
    dictionary5[agent] = pd.concat(frames)
    mediana5[agent] = pd.concat(frames).median()
    mejor5[agent] = 0

In [88]:
compare = []
for key1 in dictionary5:
    noiguales = 0
    iguales = 0
    for key2 in dictionary5:
        if key1 != key2:
            if is_unique(dictionary5[key1]) == False and is_unique(dictionary5[key2]) == False:
                S, p = mannwhitneyu(dictionary5[key1], dictionary5[key2], use_continuity=False, alternative = 'two-sided')
                if p < 0.05:
                    noiguales += 1
                    if key1 not in compare:
                        if mediana5[key1] != mediana5[key2]:
                            if mediana5[key1] > mediana5[key2]:
                                 mejor5[key1] += 1
                else:
                    iguales += 1
    compare.append(key1)
    print(key1, "Igual|No igual =>", str(iguales) + "|" + str(noiguales))

anac2012.CUHKAgent_Offering|other.AC_Uncertain|PracticeSimpleFrequencyModel_delta|BestBid_t Igual|No igual => 4|0
anac2012.CUHKAgent_Offering|other.AC_Uncertain|JaidenMeidenModel_delta_multa|BestBid_t Igual|No igual => 0|0
TheFawkes Igual|No igual => 4|0
KLH Igual|No igual => 4|0
Atlas3 Igual|No igual => 4|0
PonPokoAgent Igual|No igual => 4|0


In [89]:
for key in mejor5:
    print(key, '=>', mejor5[key])

anac2012.CUHKAgent_Offering|other.AC_Uncertain|PracticeSimpleFrequencyModel_delta|BestBid_t => 0
anac2012.CUHKAgent_Offering|other.AC_Uncertain|JaidenMeidenModel_delta_multa|BestBid_t => 0
TheFawkes => 0
KLH => 0
Atlas3 => 0
PonPokoAgent => 0
